# Frameworks

> Load and process framework data from JSON files for evaluation and analysis

In [ ]:
#| default_exp frameworks

In [ ]:
#| exports
from pathlib import Path
from rich import print
import json
from fastcore.all import *

Base type and wrapper functions for:

1. Pretty printing of evaluation framework data with truncation
2. Easy attribute-style access to evaluation framework dictionaries

In [ ]:
#| exports
class EvalDict(AttrDict):
    "A dictionary that truncates strings and lists to a maximum length for pretty printing and dot notation access of attributes"
    def __init__(self, *args, max_len=50, max_items=5, **kwargs):
        super().__init__(*args, **kwargs)
        self._max_len = max_len
        self._max_items = max_items
    
    def __repr__(self):
        def truncate(text, max_len=self._max_len):
            return text[:max_len] + "..." if len(text) > self._max_len else text
        
        def format_item(key, value):
            if isinstance(value, str):
                return f"'{key}': '{truncate(value)}'"
            elif isinstance(value, (list, dict)):
                count = len(value)
                return f"'{key}': [{count} items]"
            else:
                return f"'{key}': {value}"
        
        items = list(self.items())[:self._max_items]  
        formatted = [format_item(k, v) for k, v in items]
        suffix = ", ..." if len(self) > self._max_items else ""
        
        return f"EvalDict({{{', '.join(formatted)}{suffix}}})"

In [ ]:
#| exports
@delegates(EvalDict.__init__)
def evalify(
    data, # data to evalify
    **kwargs # kwargs to pass to EvalDict.__init__
    ):
    "Evalify a dictionary or a list of dictionaries"
    if isinstance(data, list):
        return [EvalDict(item, **kwargs) for item in data]
    else:
        return EvalDict(data, **kwargs)

For instance on the SRF Enablers and GCM to SRF lookup tables:

In [ ]:
#| eval: false
file_path = Path('files/eval_frameworks/srf_enablers.json')
srf_e = json.loads(file_path.read_text())
srf_e = evalify(srf_e, max_len=40, max_items=5); srf_e

[EvalDict({'id': '1', 'title': 'Workforce', 'description': 'IOM’s diverse and capable people are our...', 'indicators': [14 items]}),
 EvalDict({'id': '2', 'title': 'Partnership', 'description': 'Long-term partnerships built on trust me...', 'indicators': [20 items]}),
 EvalDict({'id': '3', 'title': 'Funding', 'description': 'IOM’s vision will only be realized with ...', 'indicators': [22 items]}),
 EvalDict({'id': '4', 'title': 'Data and evidence', 'description': 'IOM will be the pre-eminent source of mi...', 'indicators': [16 items]}),
 EvalDict({'id': '5', 'title': 'Learning and Innovation', 'description': 'As an innovator within the migration spa...', 'indicators': [7 items]}),
 EvalDict({'id': '6', 'title': 'Communication', 'description': 'We will take a data-driven and co-design...', 'indicators': [4 items]}),
 EvalDict({'id': '7', 'title': 'Internal systems', 'description': 'As IOM evolves, so must its organization...', 'indicators': [20 items]})]

In [ ]:
#| exports
class IOMEvalData:
    def __init__(self, base_path="./files/eval_frameworks"):
        self._data = self._load_all(Path(base_path))
    
    def _load_all(self, path): 
        return {f.stem: json.loads(f.read_text()) for f in path.ls(file_exts='.json')}
    
    @property
    def srf_crosscutting_priorities(self): return evalify(self._data['crosscutting_priorities'])
    @property
    def gcm_srf_lut(self): return evalify(self._data['gcm_to_srf_outputs'])
    @property
    def gcm_objectives(self): return evalify(self._data['gcm'])
    @property
    def srf_enablers(self): return evalify(self._data['srf_enablers'])
    @property 
    def srf_objectives(self): return evalify(self._data['srf_objectives'])
    @property
    def srf_gcm_lut(self): return evalify(self._data['srf_outputs_to_gcm'])

Here is the list of currently available files:

In [ ]:
#| eval: false
eval_data = IOMEvalData()
[o for o in dir(eval_data) if not o.startswith('_')]

['gcm_objectives',
 'gcm_srf_lut',
 'srf_crosscutting_priorities',
 'srf_enablers',
 'srf_gcm_lut',
 'srf_objectives']

By default it "pretty prints" the data:

In [ ]:
#| eval: false
eval_data.gcm_objectives

[EvalDict({'id': '1', 'title': 'Collect and utilize accurate and disaggregated dat...', 'commitment': 'We commit to strengthen the global evidence base o...', 'actions': [11 items]}),
 EvalDict({'id': '2', 'title': 'Minimize the adverse drivers and structural factor...', 'commitment': 'We commit to create conducive political, economic,...', 'actions': [12 items]}),
 EvalDict({'id': '3', 'title': 'Provide accurate and timely information at all sta...', 'commitment': 'We commit to strengthen our efforts to provide, ma...', 'actions': [5 items]}),
 EvalDict({'id': '4', 'title': 'Ensure that all migrants have proof of legal ident...', 'commitment': 'We commit to fulfil the right of all individuals t...', 'actions': [7 items]}),
 EvalDict({'id': '5', 'title': 'Enhance availability and flexibility of pathways f...', 'commitment': 'We commit to adapt options and pathways for regula...', 'actions': [10 items]}),
 EvalDict({'id': '6', 'title': 'Facilitate fair and ethical recruitment and safegu

But you can access the data via properties using the "dotted“ notation:

In [ ]:
#| eval: false
eval_data.gcm_objectives[0]

```json
EvalDict({'id': '1', 'title': 'Collect and utilize accurate and disaggregated dat...', 'commitment': 'We commit to strengthen the global evidence base o...', 'actions': [11 items]})
```

In [ ]:
#| eval: false
print(f'Title: {eval_data.gcm_objectives[0].title}')
print(f'Commitment: {eval_data.gcm_objectives[0].commitment}')
print(f'First action: {eval_data.gcm_objectives[0].actions[0]}')

Title: Collect and utilize accurate and disaggregated data as a basis for evidence-based policies

Commitment: We commit to strengthen the global evidence base on international migration by improving and investing 
in the collection, analysis and dissemination of accurate, reliable and comparable data, disaggregated by sex, age,
migration status and other characteristics relevant in national contexts, while upholding the right to privacy 
under international human rights law and protecting personal data. We further commit to ensure that this data 
fosters research, guides coherent and evidence-based policymaking and well-informed public discourse, and allows 
for effective monitoring and evaluation of the implementation of commitments over time.

First action: {'id': 'a', 'description': 'Elaborate and implement a comprehensive strategy for improving migration 
data at the local, national, regional and global levels, with the participation of all relevant stakeholders, under
the guidance of the Statistical Commission of the United Nations, by harmonizing methodologies for data collection,
and strengthening analysis and dissemination of migration-related data and indicators'}